In [ ]:
from __future__ import division
import pyomo.environ as pyomo
import pyomo.opt as opt

import pandas as pd

import matplotlib.pyplot as plt

# Welcome to the TESA programming exercise on basic dispatch LP models

## 1. Prepare the input data for the LP model

In [ ]:
T = [0,1,2,3,4,5,6,7,8,9] # set of timesteps
G = ['CCGT','GT','Coal','Lignite','Nuclear'] # generator technologies

In [ ]:
# constant price for CO2 emissions [€/t CO2]
CO2_PRICE = 20

In [ ]:
# constant length of timesteps
LENGTH = 876

In [ ]:
# convention:   tech_data[(g,'invest')] = investment costs of technology g [€/kW_el]
#               tech_data[(g,'eta_el')] = efficiency factor of technology g [1]
#               tech_data[(g,'fuel price')]= fuel price of technology g [€/MWh_thermal]
#               tech_data[(g,'other variable costs')] = other variable costs of technology g [€/MWh_el]
#               tech_data[(g,'emission factor')] = emission factor of technology g [kg CO2/MWh_thermal]
tech_data_g = {('CCGT','invest'):    500,('CCGT','RBF'):    9.8,('CCGT','eta_el'):   0.55,('CCGT','fuel price'):   28.00,('CCGT','other variable costs'):    5.23,('CCGT','emissions'):   204.8, 
               ('GT','invest'):      450,('GT','RBF'):      9.8,('GT','eta_el'):     0.37,('GT','fuel price'):     28.00,('GT','other variable costs'):      3.60,('GT','emissions'):     204.8, 
               ('Coal','invest'):   1300,('Coal','RBF'):   11.3,('Coal','eta_el'):   0.42,('Coal','fuel price'):   10.47,('Coal','other variable costs'):    9.70,('Coal','emissions'):   342.0,
               ('Lignite','invest'):1400,('Lignite','RBF'):11.3,('Lignite','eta_el'):0.39,('Lignite','fuel price'): 7.50,('Lignite','other variable costs'):13.44,('Lignite','emissions'):400.0, 
               ('Nuclear','invest'):3000,('Nuclear','RBF'):11.9,('Nuclear','eta_el'):0.35,('Nuclear','fuel price'): 3.47,('Nuclear','other variable costs'): 7.46,('Nuclear','emissions'):  0.0}

In [ ]:
# convention: load_t[t] = constant load during timestep t [MW]
load_t = [73409,69648,66632,63454,59526,55367,51970,49045,46030,41466]

In [ ]:
# peak load [MW]
peak_load = 79487

## 2. Build the LP model

In [ ]:
model = pyomo.ConcreteModel()

### 2.1 Define Sets

In [ ]:
model.T = pyomo.Set(initialize=T)
model.G = pyomo.Set(initialize=G)

### 2.2 Define Variables

In [ ]:
model.x_g_t = pyomo.Var(model.G, model.T, domain=pyomo.NonNegativeReals)    # dispatched power of technology g during hour t [MW]
model.y_g = pyomo.Var(model.G, domain=pyomo.NonNegativeReals)               # installed capacity per technology g [MW]

### 2.3 Define Constraints

In [ ]:
# cover demand
def define_demand_restriction(model, t):
    return sum(model.x_g_t[g, t] for g in model.G) == load_t[t]
model.demand_restriction = pyomo.Constraint(model.T, rule=define_demand_restriction)

In [ ]:
# cover peak load
def define_peak_load_restriction(model):
    return sum(model.y_g[g] for g in model.G) == peak_load
model.peak_load_restriction = pyomo.Constraint(rule=define_peak_load_restriction)

In [ ]:
# capacity restriciton and variable connection
def define_capacity_restriciton(model, g, t):
    return model.x_g_t[g,t] <= model.y_g[g]
model.capacity_restriciton = pyomo.Constraint(model.G, model.T, rule=define_capacity_restriciton)

In [ ]:
# no use of nuclear power plants
def define_nuclear_restriciton(model):
    return model.y_g['Nuclear'] == 0
model.nuclear_restriciton = pyomo.Constraint(rule=define_nuclear_restriciton)

### 2.4 Define Objective Function

In [ ]:
# define objective function (i.e. dispatch costs)
def define_objective_function(model):
    return sum(
                tech_data_g[(g,'invest')] * 1000 / tech_data_g[(g,'RBF')] * \
                model.y_g[g] 
                for g in model.G
              ) + \
           sum(
                (
                    (tech_data_g[(g,'fuel price')] + tech_data_g[(g,'emissions')] * (CO2_PRICE/1000)) / 
                    tech_data_g[(g,'eta_el')] + \
                    tech_data_g[(g,'other variable costs')] \
                ) * \
                model.x_g_t[g,t] * LENGTH 
                for g in model.G for t in model.T
              )
model.Obj = pyomo.Objective(rule=define_objective_function, sense=pyomo.minimize)

### 2.5 Write LP to File

In [ ]:
model.write('output/invest/08_tesa_uebung_LP2_investition_loesung.lp', io_options={'symbolic_solver_labels':True})

### 2.6 Initialize the storage of dual variables of constraints

In [ ]:
model.dual = pyomo.Suffix(direction=pyomo.Suffix.IMPORT)

## 3. Solve the LP model

In [ ]:
optimizer = opt.SolverFactory('glpk')
solved_model = optimizer.solve(model, tee=True)

## 4. Get the results and statistics of the solved LP model

### 4.1 Print optimal objective value

In [ ]:
print("Optimal value: %.2f Mio. €" % (round(model.Obj.expr()/1e6,2)))

### 4.2 Print optimal invests

In [ ]:
EPS = 1e-6

for g in G:
    if pyomo.value(model.y_g[g]) > EPS: 
        print('Install %i MW of technology %s' % (pyomo.value(model.y_g[g]),g))

### 4.3 Print optimal dispatch path

In [ ]:
for g in G:
    for t in T:
        if pyomo.value(model.x_g_t[g,t]) > EPS:
            print("Dispatch %i TWh of technology %s in timestep %s" % (pyomo.value(model.x_g_t[g,t])*LENGTH/1e6,g,t))        

### 4.4 Calculate full load hours

In [ ]:
full_load_hours = {}
for g in G:
    if pyomo.value(model.y_g[g]) > EPS:
        full_load_hours[g] = sum(pyomo.value(model.x_g_t[g,t]) * LENGTH for t in T) / pyomo.value(model.y_g[g])
        print('Full load hours of technology %s: %i h' % (g,full_load_hours[g]))
    else:
        print('Technology %s is not installed' % g)

### 4.5 Calculate total emissions

In [ ]:
total_emissions = {}
for g in G:
    total_emissions[g] = sum(pyomo.value(model.x_g_t[g,t]) * LENGTH * (tech_data_g[(g,'emissions')]) / tech_data_g[(g,'eta_el')] for t in T) / 10**9 # unit: Mt
    print('Total emissions of technology %s: %i Mt' % (g,total_emissions[g]))

print('Overall emissions: %i Mt' % sum(total_emissions[g] for g in G))

### 4.6 Print electricity prices as shadow prices of the demand constraint

In [ ]:
dual_values = pd.Series(list(model.dual.values()), index=pd.Index(list(model.dual.keys())))
electricity_shadow_prices = pd.Series(list(model.demand_restriction.values()), index=pd.Index(list(model.demand_restriction.keys()))).map(dual_values).divide(LENGTH)

In [ ]:
for t in T:
    print("Electricity price during t= "+str(t)+": "+str(round(electricity_shadow_prices[t],1))+' €/MWh')

### 4.7 calculate total profit per technology --> contribution - fixed costs

In [ ]:
variable_costs = {}
contribution = {}
fixed_costs ={}
profit = {}
for g in G:
    variable_costs[g]       = (tech_data_g[(g,'fuel price')] + tech_data_g[(g,'emissions')]*CO2_PRICE/1000)/tech_data_g[(g,'eta_el')] + tech_data_g[(g,'other variable costs')]
    contribution[g]         = sum ((electricity_shadow_prices[t]-variable_costs[g])*pyomo.value(model.x_g_t[g,t])*LENGTH for t in T)
    fixed_costs[g]          = pyomo.value(model.y_g[g])*tech_data_g[(g,'invest')]*1000/tech_data_g[(g,'RBF')]
    profit[g]               = (contribution[g]-fixed_costs[g])/1e6
    print('Total profit of technology %s: %i Mio €' % (g,profit[g]))

## 5. Visualize results

### 5.1 Create pie chart of installed capacities

In [ ]:
sizes = []
labels = []
for g in G:
    if pyomo.value(model.y_g[g]) > EPS: 
        sizes.append(pyomo.value(model.y_g[g]))
        labels.append(g)
total = sum(sizes)

In [ ]:
fig1, ax1 = plt.subplots()

ax1.pie(sizes,labels=labels, autopct=lambda p: '{:.0f}'.format(p * total / 100)) # autopct is used for automatic labelling, no need to understand all coding details here
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

### 5.2 Create line chart of dispatched power

In [ ]:
dispatch_g = {g: [] for g in G}
for t in T:
    for g in G:
        dispatch_g[g].append(pyomo.value(model.x_g_t[g,t]))

In [ ]:
fig2, ax2 = plt.subplots()

for g in G:
    ax2.plot(T, dispatch_g[g])
ax2.legend(G)
ax2.xaxis.set_ticks(T)
ax2.set_ylabel('Dispatched Power [MW]')